# Multinomial Naive Bayes 

Here I will make every feature categorical using OneHotEnconding for Categorical, and Discretization for 
Continuous and then fit Multinomial Naive Bayes on the the data

[Spam Filtering with Naive Bayes – Which Naive Bayes? ∗](http://www2.aueb.gr/users/ion/docs/ceas2006_paper.pdf)

In [1]:
import os
import sys
sys.path.append("../")

from titansurv.preprocessing import DecisionTreeDiscretizer
from titansurv.preprocessing.transformers import NaNDropper
from sklearn.naive_bayes import MultinomialNB

In [2]:
import pandas as pd

In [3]:
df = pd.read_csv('../data/raw/train.csv').drop('PassengerId', axis=1)
dfX = df.drop('Survived', axis=1)
dfy = df.Survived

In [4]:
from sklearn.model_selection import train_test_split

In [5]:
dfX.head()

,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


## Preprocessing_pre

In [6]:
from sklearn.pipeline import Pipeline

In [7]:
pre1 = Pipeline([
    ('nan_drpr', NaNDropper(['Embarked']))
])

dfX, dfy = pre1.fit_transform(dfX, dfy)

In [ ]:
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer

In [8]:
pre2 = Pipeline([
    ('imp', SimpleImputer()),
    ('scaler', StandardScaler()),
    ('discretizer', DecisionTreeDiscretizer(encoder='ohe'))
], 'passthrough')



precomb2 = ColumnTransformer([
    ('clmn_drpr', 'drop', ['Name', 'Ticket', 'Cabin']),
    ('enc', OneHotEncoder(drop='first'), ['Sex', 'Embarked']),
    ('imp_scaler', pre2, ['Age', 'Fare'])
], 'passthrough')

In [58]:
precomb2.fit_transform(dfX, dfy).shape

(889, 24)

In [59]:
from titansurv.utils import print_params

In [60]:
print_params(precomb2)

['n_jobs',
 'remainder',
 'sparse_threshold',
 'transformer_weights',
 'transformers',
 'verbose',
 'clmn_drpr',
 'enc',
 'imp_scaler',
 'enc__categories',
 'enc__drop',
 'enc__dtype',
 'enc__handle_unknown',
 'enc__sparse',
 'imp_scaler__memory',
 'imp_scaler__steps',
 'imp_scaler__verbose',
 'imp_scaler__imp',
 'imp_scaler__scaler',
 'imp_scaler__discretizer',
 'imp_scaler__imp__add_indicator',
 'imp_scaler__imp__copy',
 'imp_scaler__imp__fill_value',
 'imp_scaler__imp__missing_values',
 'imp_scaler__imp__strategy',
 'imp_scaler__imp__verbose',
 'imp_scaler__scaler__copy',
 'imp_scaler__scaler__with_mean',
 'imp_scaler__scaler__with_std',
 'imp_scaler__discretizer__ccp_alpha',
 'imp_scaler__discretizer__class_weight',
 'imp_scaler__discretizer__criterion',
 'imp_scaler__discretizer__encoder',
 'imp_scaler__discretizer__max_depth',
 'imp_scaler__discretizer__max_features',
 'imp_scaler__discretizer__max_leaf_nodes',
 'imp_scaler__discretizer__min_impurity_decrease',
 'imp_scaler__disc

In [61]:
from sklearn.naive_bayes import MultinomialNB, BernoulliNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold

In [62]:
pipe = Pipeline([
    ('preprocessing', precomb2),
    ('clf', BernoulliNB())
])

In [63]:
cross_val_score(pipe, dfX, dfy)

array([0.66853933, 0.66292135, 0.64606742, 0.69662921, 0.64971751])

In [16]:
print_params(pipe)

['memory',
 'steps',
 'verbose',
 'preprocessing',
 'clf',
 'preprocessing__n_jobs',
 'preprocessing__remainder',
 'preprocessing__sparse_threshold',
 'preprocessing__transformer_weights',
 'preprocessing__transformers',
 'preprocessing__verbose',
 'preprocessing__clmn_drpr',
 'preprocessing__enc',
 'preprocessing__imp_scaler',
 'preprocessing__enc__categories',
 'preprocessing__enc__drop',
 'preprocessing__enc__dtype',
 'preprocessing__enc__handle_unknown',
 'preprocessing__enc__sparse',
 'preprocessing__imp_scaler__memory',
 'preprocessing__imp_scaler__steps',
 'preprocessing__imp_scaler__verbose',
 'preprocessing__imp_scaler__imp',
 'preprocessing__imp_scaler__scaler',
 'preprocessing__imp_scaler__discretizer',
 'preprocessing__imp_scaler__imp__add_indicator',
 'preprocessing__imp_scaler__imp__copy',
 'preprocessing__imp_scaler__imp__fill_value',
 'preprocessing__imp_scaler__imp__missing_values',
 'preprocessing__imp_scaler__imp__strategy',
 'preprocessing__imp_scaler__imp__verbose'

In [17]:
from sklearn.model_selection import GridSearchCV

In [18]:
# Discretizer hyperparameters
min_samples_split = [2, 5, 10, 20]
max_depth = [None, 5, 10, 25, 50, 100]
min_samples_leaf = [1, 2, 5, 10]

# Multinomial Naive Bayes 
alpha = [1, 0.8, 0.4, 0.1, 0.01]


param_grid = {
    'preprocessing__imp_scaler__discretizer': [DecisionTreeDiscretizer()],
    'preprocessing__imp_scaler__discretizer__min_samples_split': min_samples_split,
    'preprocessing__imp_scaler__discretizer__max_depth': max_depth,
    'preprocessing__imp_scaler__discretizer__min_samples_leaf': min_samples_leaf,
    'clf__alpha': alpha
}

In [19]:
%%time
grid = GridSearchCV(pipe, param_grid)
grid.fit(dfX, dfy)

/home/abhi/miniconda3/envs/titansurv/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:532: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
ValueError: bad input shape (711, 2)

  warnings.warn("Estimator fit failed. The score on this train-test"
/home/abhi/miniconda3/envs/titansurv/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:532: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
ValueError: bad input shape (711, 2)

  warnings.warn("Estimator fit failed. The score on this train-test"
/home/abhi/miniconda3/envs/titansurv/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:532: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
ValueError: bad input shape (711, 2)

  warnings.warn("Estimator fit 

/home/abhi/miniconda3/envs/titansurv/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:532: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
ValueError: bad input shape (712, 2)

  warnings.warn("Estimator fit failed. The score on this train-test"
/home/abhi/miniconda3/envs/titansurv/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:532: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
ValueError: bad input shape (711, 2)

  warnings.warn("Estimator fit failed. The score on this train-test"
/home/abhi/miniconda3/envs/titansurv/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:532: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
ValueError: bad input shape (711, 2)

  warnings.warn("Estimator fit 

/home/abhi/miniconda3/envs/titansurv/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:532: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
ValueError: bad input shape (712, 2)

  warnings.warn("Estimator fit failed. The score on this train-test"
/home/abhi/miniconda3/envs/titansurv/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:532: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
ValueError: bad input shape (711, 2)

  warnings.warn("Estimator fit failed. The score on this train-test"
/home/abhi/miniconda3/envs/titansurv/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:532: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
ValueError: bad input shape (711, 2)

  warnings.warn("Estimator fit 

/home/abhi/miniconda3/envs/titansurv/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:532: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
ValueError: bad input shape (712, 2)

  warnings.warn("Estimator fit failed. The score on this train-test"
/home/abhi/miniconda3/envs/titansurv/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:532: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
ValueError: bad input shape (711, 2)

  warnings.warn("Estimator fit failed. The score on this train-test"
/home/abhi/miniconda3/envs/titansurv/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:532: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
ValueError: bad input shape (711, 2)

  warnings.warn("Estimator fit 

/home/abhi/miniconda3/envs/titansurv/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:532: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
ValueError: bad input shape (711, 2)

  warnings.warn("Estimator fit failed. The score on this train-test"
/home/abhi/miniconda3/envs/titansurv/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:532: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
ValueError: bad input shape (711, 2)

  warnings.warn("Estimator fit failed. The score on this train-test"
/home/abhi/miniconda3/envs/titansurv/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:532: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
ValueError: bad input shape (711, 2)

  warnings.warn("Estimator fit 

...

/home/abhi/miniconda3/envs/titansurv/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:532: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
ValueError: bad input shape (711, 2)

  warnings.warn("Estimator fit failed. The score on this train-test"
/home/abhi/miniconda3/envs/titansurv/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:532: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
ValueError: bad input shape (711, 2)

  warnings.warn("Estimator fit failed. The score on this train-test"
/home/abhi/miniconda3/envs/titansurv/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:532: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
ValueError: bad input shape (711, 2)

  warnings.warn("Estimator fit 

/home/abhi/miniconda3/envs/titansurv/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:532: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
ValueError: bad input shape (711, 2)

  warnings.warn("Estimator fit failed. The score on this train-test"
/home/abhi/miniconda3/envs/titansurv/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:532: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
ValueError: bad input shape (711, 2)

  warnings.warn("Estimator fit failed. The score on this train-test"
/home/abhi/miniconda3/envs/titansurv/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:532: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
ValueError: bad input shape (711, 2)

  warnings.warn("Estimator fit 

/home/abhi/miniconda3/envs/titansurv/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:532: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
ValueError: bad input shape (711, 2)

  warnings.warn("Estimator fit failed. The score on this train-test"
/home/abhi/miniconda3/envs/titansurv/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:532: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
ValueError: bad input shape (711, 2)

  warnings.warn("Estimator fit failed. The score on this train-test"
/home/abhi/miniconda3/envs/titansurv/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:532: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
ValueError: bad input shape (712, 2)

  warnings.warn("Estimator fit 

/home/abhi/miniconda3/envs/titansurv/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:532: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
ValueError: bad input shape (711, 2)

  warnings.warn("Estimator fit failed. The score on this train-test"
/home/abhi/miniconda3/envs/titansurv/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:532: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
ValueError: bad input shape (711, 2)

  warnings.warn("Estimator fit failed. The score on this train-test"
/home/abhi/miniconda3/envs/titansurv/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:532: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
ValueError: bad input shape (711, 2)

  warnings.warn("Estimator fit 

KeyboardInterrupt: 

In [71]:
grid.best_score_

0.7085951882181172

In [64]:
grid.best_estimator_

Pipeline(memory=None,
         steps=[('preprocessing',
                 ColumnTransformer(n_jobs=None, remainder='passthrough',
                                   sparse_threshold=0.3,
                                   transformer_weights=None,
                                   transformers=[('clmn_drpr', 'drop',
                                                  ['Name', 'Ticket', 'Cabin']),
                                                 ('enc',
                                                  OneHotEncoder(categories='auto',
                                                                drop='first',
                                                                dtype=<class 'numpy.float64'>,
                                                                handle_unknown='error',
                                                                sparse=True),
                                                  ['Sex', 'Embarked']),
                                                 ('

In [100]:
from sklearn.model_selection import cross_val_score

In [101]:
precomb2.fit_transform(dfX, dfy)

array([[1., 0., 1., ..., 3., 1., 0.],
       [0., 0., 0., ..., 1., 1., 0.],
       [0., 0., 1., ..., 3., 0., 0.],
       ...,
       [0., 0., 1., ..., 3., 1., 2.],
       [1., 0., 0., ..., 1., 0., 0.],
       [1., 1., 0., ..., 3., 0., 0.]])

In [105]:
cross_val_score(MultinomialNB(), precomb2.fit_transform(dfX, dfy), dfy, cv=5)

array([0.92696629, 0.94382022, 0.96629213, 0.93258427, 0.92655367])